In [29]:
from struct import unpack
import numpy as np
import matplotlib.pyplot as plt

ImportError: No module named 'matplotlib'

In [8]:
with open('../train/audio/bed/00176480_nohash_0.wav', 'rb') as wav_file:
    # Top chunk
    
    ### Obtain chunk id
    chunk_id = wav_file.read(4)
    assert chunk_id == b'RIFF'

    ### Obtain chunk size
    # '<' : LE (is LE instead of BE, in which case chunk_id == 'RIFX')
    # 'I' : 4B unsigned integer
    chunk_size = unpack('<I', wav_file.read(4))[0]
    print("total chunk size : ", chunk_size)
    
    ### Obtain wav format and verify that it is 'WAVE'
    wav_format = wav_file.read(4)
    assert wav_format == b'WAVE'
    
    
    # Second chunk
    
    ### Chunk 1 information (ID, size)
    sub_chunk_1_id = wav_file.read(4)
    assert sub_chunk_1_id == b'fmt '
    sub_chunk_1_size = unpack('<I', wav_file.read(4))[0]
    print("subchunk 1 size :", sub_chunk_1_size)
    
    ### Audio format
    audio_format = unpack('<h', wav_file.read(2))[0]
    if audio_format == 1:
        print("audio_format : PCM")
        
    ### number of channels
    num_channels = unpack('<h', wav_file.read(2))[0]
    if num_channels == 1:
        print("channels : Mono")
    elif num_channels == 2:
        print("channels : Stereo")
    else:
        print("num_channels : ", num_channels)
        
    ### Sampling rate
    sample_rate = unpack('<I', wav_file.read(4))[0]
    print("sampling rate : ", sample_rate)
    
    ### Byte rate : # of bytes for each interval
    byte_rate = unpack('<I', wav_file.read(4))[0]
    print("byte rate : ", byte_rate)
    
    ### Block align : # of bytes per each sample
    block_align = unpack('<h', wav_file.read(2))[0]

    ### Obtain Bits-per-sample value
    #wav_file.seek(34)
    bits_per_sample = unpack('<h', wav_file.read(2))[0]
    bytes_per_sample = int(bits_per_sample / 8)
    print("bytes / sample : ", bytes_per_sample)
    
    assert byte_rate == sample_rate * num_channels * bytes_per_sample
    assert block_align == num_channels * bytes_per_sample

    
    # Third chunk
    
    ### Subchunk 2 information
    sub_chunk_2_id = wav_file.read(4)
    sub_chunk_2_size = unpack('<I',wav_file.read(4))[0]
    print("subchunk 2 size : ", sub_chunk_2_size)
    
    ### Summary
    length_time = sub_chunk_2_size / byte_rate
    length_interval = 100    # 100 ms / interval
    print("Length of input (s) : ", length_time)
    
    ### Obtain the actual samples
    samples = []
    
    bytes_per_timestep = bytes_per_sample * num_channels
    timesteps_per_100ms = int(sample_rate / int(1000 / length_interval))
    num_intervals = int(length_time * int(1000 / length_interval))
    print(num_intervals, timesteps_per_100ms, bytes_per_timestep)
    
    for interval in range(num_intervals):
        interval_samples = []
        for _ in range(timesteps_per_100ms):
            timestep_sample = []
            for _ in range(num_channels):
                frag = wav_file.read(bytes_per_sample)
                if bytes_per_sample == 2:
                    timestep_sample.append(unpack('<h', frag)[0])
                elif bytes_per_sample == 1:
                    timestep_sample.append(unpack('<b', frag)[0])
            interval_samples.append(timestep_sample)
        samples.append(interval_samples)

total chunk size :  1437766
subchunk 1 size : 16
audio_format : PCM
channels : Stereo
sampling rate :  44100
byte rate :  176400
bytes / sample :  2
subchunk 2 size :  26
Length of input (s) :  0.00014739229024943312
0 4410 4


In [9]:
print(len(samples))
print(len(samples[0]))
print(samples[0])

0


IndexError: list index out of range

In [21]:
with open('../test_record.wav', 'rb') as wav_file:
    # Top chunk - 'RIFF'
    
    ### Obtain chunk id
    chunk_id = wav_file.read(4)
    assert chunk_id == b'RIFF'

    ### Obtain chunk size
    # '<' : LE (is LE instead of BE, in which case chunk_id == 'RIFX')
    # 'I' : 4B unsigned integer
    chunk_size = unpack('<I', wav_file.read(4))[0]
    print("total chunk size : ", chunk_size)
    
    ### Obtain wav format and verify that it is 'WAVE'
    wav_format = wav_file.read(4)
    assert wav_format == b'WAVE'
    
    
    # First chunk - 'fmt'
    
    ### Chunk 1 information (ID, size)
    sub_chunk_1_id = wav_file.read(4)
    assert sub_chunk_1_id == b'fmt '
    sub_chunk_1_size = unpack('<I', wav_file.read(4))[0]
    print("subchunk 1 size :", sub_chunk_1_size)
    
    ### Audio format
    audio_format = unpack('<h', wav_file.read(2))[0]
    if audio_format == 1:
        print("audio_format : PCM")
        
    ### number of channels
    num_channels = unpack('<h', wav_file.read(2))[0]
    if num_channels == 1:
        print("channels : Mono")
    elif num_channels == 2:
        print("channels : Stereo")
    else:
        print("num_channels : ", num_channels)
        
    ### Sampling rate
    sample_rate = unpack('<I', wav_file.read(4))[0]
    print("sampling rate : ", sample_rate)
    
    ### Byte rate : # of bytes for each interval
    byte_rate = unpack('<I', wav_file.read(4))[0]
    print("byte rate : ", byte_rate)
    
    ### Block align : # of bytes per each sample
    block_align = unpack('<h', wav_file.read(2))[0]

    ### Obtain Bits-per-sample value
    #wav_file.seek(34)
    bits_per_sample = unpack('<h', wav_file.read(2))[0]
    bytes_per_sample = int(bits_per_sample / 8)
    print("bytes / sample : ", bytes_per_sample)
    
    assert byte_rate == sample_rate * num_channels * bytes_per_sample
    assert block_align == num_channels * bytes_per_sample

    
    # Second chunk - 'LIST'
    
    ### Subchunk 2 information
    wav_file.seek(36)
    sub_chunk_2_id = wav_file.read(4)
    sub_chunk_2_size = unpack('<I',wav_file.read(4))[0]
    print("subchunk 2 id : ", sub_chunk_2_id)
    print("subchunk 2 size : ", sub_chunk_2_size)
    
    
    # Third chunk - 'DATA'
    
    ### Information
    wav_file.seek(70)
    sub_chunk_3_id = wav_file.read(4)
    sub_chunk_3_size = unpack('<I',wav_file.read(4))[0]
    print("subchunk 3 id : ", sub_chunk_3_id)
    print("subchunk 3 size : ", sub_chunk_3_size)
    
    ### Summary
    length_time = sub_chunk_3_size / byte_rate
    length_interval = 100    # 100 ms / interval
    print("Length of input (s) : ", length_time)
    
    ### Obtain the actual samples
    samples = []
    
    bytes_per_timestep = bytes_per_sample * num_channels
    timesteps_per_100ms = int(sample_rate / int(1000 / length_interval))
    num_intervals = int(length_time * int(1000 / length_interval))
    print(num_intervals, timesteps_per_100ms, bytes_per_timestep)
    
    for interval in range(num_intervals):
        interval_samples = []
        for _ in range(timesteps_per_100ms):
            timestep_sample = []
            for _ in range(num_channels):
                frag = wav_file.read(bytes_per_sample)
                if bytes_per_sample == 2:
                    timestep_sample.append(unpack('<h', frag)[0])
                elif bytes_per_sample == 1:
                    timestep_sample.append(unpack('<b', frag)[0])
            interval_samples.append(timestep_sample)
        samples.append(interval_samples)

total chunk size :  1437766
subchunk 1 size : 16
audio_format : PCM
channels : Stereo
sampling rate :  44100
byte rate :  176400
bytes / sample :  2
subchunk 2 id :  b'LIST'
subchunk 2 size :  26
subchunk 3 id :  b'data'
subchunk 3 size :  1437696
Length of input (s) :  8.150204081632653
81 4410 4


In [24]:
print(len(samples))
print(len(samples[40]))
print(samples[40])

81
4410
[[-531, -531], [-544, -544], [-523, -523], [-522, -522], [-526, -526], [-515, -515], [-523, -523], [-522, -522], [-532, -532], [-550, -550], [-565, -565], [-595, -595], [-604, -604], [-627, -627], [-638, -638], [-634, -634], [-649, -649], [-634, -634], [-634, -634], [-637, -637], [-622, -622], [-612, -612], [-611, -611], [-635, -635], [-623, -623], [-634, -634], [-670, -670], [-674, -674], [-696, -696], [-708, -708], [-714, -714], [-726, -726], [-709, -709], [-708, -708], [-714, -714], [-684, -684], [-684, -684], [-693, -693], [-688, -688], [-696, -696], [-712, -712], [-719, -719], [-734, -734], [-771, -771], [-776, -776], [-797, -797], [-832, -832], [-837, -837], [-852, -852], [-875, -875], [-883, -883], [-895, -895], [-919, -919], [-924, -924], [-947, -947], [-995, -995], [-1036, -1036], [-1070, -1070], [-1148, -1148], [-1219, -1219], [-1276, -1276], [-1359, -1359], [-1411, -1411], [-1465, -1465], [-1513, -1513], [-1554, -1554], [-1558, -1558], [-1557, -1557], [-1560, -1560],

In [26]:
data = np.array(samples)
print(data.shape)

(81, 4410, 2)
